In [8]:
import yaml
import itertools
from copy import deepcopy


with open('sweep_definition.yaml', 'r') as file:
    sweep_config = yaml.safe_load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'sweep_definition.yaml'

In [9]:

def flatten_dict(variant_dict):
    variant_name = list(variant_dict.keys())[0]
    variant_values = variant_dict[variant_name]
    result_dict = {'variant_name': variant_name}
    result_dict.update(variant_values)
    return result_dict

def cartesian_product(*lists):
    cartesian_product = []
    for items in itertools.product(*lists):
        merged_dict = {}
        for item in items:
            merged_dict.update(item)
        cartesian_product.append(merged_dict)
    return cartesian_product

def duplicate_dict(dictionary):
    result = [dictionary]
    for key, value in dictionary.items():
        if isinstance(value, list):
            temp_result = []
            for item in value:
                for res in result:
                    temp_dict = res.copy()
                    temp_dict[key] = item
                    temp_result.append(temp_dict)
            result = temp_result
    return result



search_space =[]
for node in sweep_config.get('search_space'):
    if node in ['llm','embedding','vector_store']:
        grid = []
        node_values = sweep_config.get('search_space').get(node)
        for variant in node_values:
            step_dict ={}
            step_dict[node] = flatten_dict(variant)
            grid.append(step_dict)
        grid_final = []
        for step in grid:
            
            node_id = list(step.keys())[0]    
            fixed = {key:value for key,value in step.get(node_id).items() if not isinstance(value, dict)}
            for key, value in step.get(node_id).items():
                option_list =[]
                if isinstance(value, dict):
                    options = duplicate_dict(value)
                    for option in options:
                        f = deepcopy(fixed)
                        f[key] = option.get('values')
                        option_list.append(f)    
            
            if len(option_list)>1:
                for opt in option_list:
                    grid_final.append({node_id: opt})
            else:
                grid_final.append({node_id: fixed})
        search_space.append(grid_final)

NameError: name 'sweep_config' is not defined

In [10]:
grid_search_steps = cartesian_product(*search_space)

In [4]:
grid_search_steps[3]

{'llm': {'variant_name': 'mistral',
  'family': 'MISTRAL',
  'connection': 'mistral-large-maas'},
 'embedding': {'variant_name': 'aoai',
  'family': 'AZUREOPENAI',
  'connection': 'aoai',
  'deployment': 'text-embedding-ada-002'}}

In [5]:
import json
import subprocess

file_path = "./flow_template/cookiecutter_template.json"
with open(file_path, "r") as file:
    data = json.load(file)



In [6]:
from unique_names_generator import get_random_name
import random
import string


def generate_step_id():
    name = get_random_name(separator="_", style="lowercase")
    unique_id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    return name+'_'+ unique_id

# Generate a unique ID
unique_id = generate_step_id()
print(unique_id)


def create_cookiecutter(data, grid_step):
    data_dict = deepcopy(data)
    for node_id in data_dict:
        if node_id in ['llm','embedding','vector_store'] and node_id in grid_step.keys():
            data_dict[node_id]=grid_step[node_id]
        elif node_id in ['llm','embedding','vector_store'] and node_id not in grid_step.keys():
            del data_dict[node_id]
    return data_dict

white_kiwi_usxrw2ck


In [10]:
import os
file_path = "./flow_template/cookiecutter.json"

for i, search_step_dict in enumerate(grid_search_steps):
    directory = 'flow_versions'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    step_dict = create_cookiecutter(data, search_step_dict)
    step_dict["flow_name"] = f"grid_step_{i}"
    
    with open(file_path, "w") as file:
        json.dump(step_dict, file, indent=4)
    
    subprocess.run(["cd ./flow_versions && cookiecutter ../flow_template --no-input --skip-if-file-exists"], shell=True)


In [6]:
!pip install promptflow

  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached ruamel.yaml-0.18.6-py3-none-any.whl.metadata (23 kB)
  Using cached cryptography-42.0.5-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.3 kB)
  Using cached Flask_Cors-4.0.0-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached flask_restx-1.3.0-py2.py3-none-any.whl.metadata (9.3 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached keyring-24.3.1-py3-none-any.whl.metadata (20 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.24.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached waitress-2.1.2-py3-none-any.whl.metadata (7.0 kB)
  Using cached opentelemetry_sdk-1.24.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached tiktoken-0.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached fixedint-0.1.6-py3-none-any.whl.metadata (4.8 kB)
  Using cached opentelemetr

In [128]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from promptflow.azure import PFClient


credential = DefaultAzureCredential()
  
# Get a handle to workspace, it will use config.json in current and parent directory.
pf = PFClient.from_config(
    credential=credential
    )

Found the config file in: /home/vhoudebine/config.json


In [7]:
import os

def create_and_run_flow(i):
    # Apply your function to the dictionary here
    flow = os.path.join("./flow_versions", f"rag_flow_grid_step_{i}")
    data ="./evaluation_data/data.jsonl"
    print(f"Creating run for {flow}")
    # create run
    base_run = pf.run(
        flow=flow,
        data=data,
        column_mapping={
        "question": "${data.question}"
        }
    )
    print(f"Created run {base_run.name}")
    return base_run


# Create a thread for each dictionary
runs = []
for i, search_step_dict in enumerate(grid_search_steps)[:1]:
    run = create_and_run_flow(i)
    runs.append(run)

NameError: name 'grid_search_steps' is not defined

In [4]:
pf.runs.get('rag_flow_grid_step_2_variant_0_20240415_170950_374378').status

'Failed'

In [5]:
run = pf.runs.get('rag_flow_grid_step_2_variant_0_20240415_170950_374378')

In [6]:
# Get run results and store them
run.name

'rag_flow_grid_step_2_variant_0_20240415_170950_374378'

In [11]:
['e','b'].count('b')

1

In [4]:

import sys
sys.path.append('..')
from flowgrid import utils

#utils.get_run_info('rag_flow_grid_step_2_variant_0_20240415_170950_374378')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [2]:
utils.get_output_asset_id('rag_flow_grid_step_2_variant_0_20240415_170950_374378')

TypeError: get_output_asset_id() missing 1 required positional argument: 'asset_name'

In [5]:

utils.get_run_info('rag_flow_grid_step_2_variant_0_20240415_170950_374378')

('rag_flow_grid_step_2_variant_0_20240415_170950_374378', None)

In [6]:
run = pf.runs.get('rag_flow_grid_step_2_variant_0_20240415_170950_374378')

In [7]:
run.properties

{'azureml.promptflow.runtime_name': 'automatic',
 'azureml.promptflow.inputs_mapping': '{"question":"${data.question}"}',
 'azureml.promptflow.disable_trace': 'false',
 'azureml.promptflow.session_id': '7b022dbbad3617f8db5c31906032b7f3ec900b3306cecc4a',
 'azureml.promptflow.definition_file_name': 'flow.dag.yaml',
 'azureml.promptflow.flow_lineage_id': '536aaae1360552919ad7489ec59e4ada907dc5606d3b3fe5bd11747b63999c0d',
 'azureml.promptflow.flow_definition_datastore_name': 'workspaceblobstore',
 'azureml.promptflow.flow_definition_blob_path': 'LocalUpload/d620a9ee65156b304f000ec623a87e7f/rag_flow_grid_step_2/flow.dag.yaml',
 '_azureml.evaluation_run': 'promptflow.BatchRun',
 'azureml.promptflow.snapshot_id': '86934ceb-ad40-4838-9f0d-36002686d6c4',
 'azureml.promptflow.runtime_version': '20240411.v4'}

In [89]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

tenant_id = "16b3c013-d300-468d-ac64-7eda0820b6d3"
subscription_id = "6c065ea7-65cd-4a34-8e2a-3e21ad4a8e9f"
resource_group = "vince-rg"
workspace_name = "vince-dev"

interactive_auth = InteractiveLoginAuthentication()
ws = Workspace.get(workspace_name, subscription_id=subscription_id, resource_group=resource_group, auth=interactive_auth)
region = ws.location
default_datastore = ws.get_default_datastore()

In [91]:
region

'eastus'

In [78]:
from azureml.core.authentication import AzureCliAuthentication

cli_auth = AzureCliAuthentication()

ws = Workspace(subscription_id=subscription_id,
               resource_group=resource_group,
               workspace_name= workspace_name,
               auth=cli_auth)

print("Found workspace {} at location {}".format(ws.name, ws.location))

AuthenticationException: AuthenticationException:
	Message: azure-cli package is not installed. AzureCliAuthentication requires azure-cli>=2.30.0 to be installed in the same python environment where azureml-sdk is installed.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
            "code": "Authentication"
        },
        "message": "azure-cli package is not installed. AzureCliAuthentication requires azure-cli>=2.30.0 to be installed in the same python environment where azureml-sdk is installed."
    }
}

In [67]:
import os 
subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID")
resource_group = os.environ.get("AZURE_RESOURCE_GROUP")
workspace_name = os.environ.get("AZURE_WORKSPACE_NAME")

In [68]:
workspace_name

In [66]:
!source ../.env

In [73]:
from azure.ai.ml import MLClient

ml_client = MLClient(
    credential=credential,
    subscription_id = subscription_id,
    resource_group_name = resource_group,
    workspace_name = workspace_name,
)

ml_client.datastores.get_default()

AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'workspaceblobstore', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/6c065ea7-65cd-4a34-8e2a-3e21ad4a8e9f/resourceGroups/vince-rg/providers/Microsoft.MachineLearningServices/workspaces/vince-dev/datastores/workspaceblobstore', 'Resource__source_path': '', 'base_path': '/home/vhoudebine/projects/pf-models-evaluation/notebooks', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fbfdc0cb050>, 'credentials': {'type': 'account_key'}, 'container_name': 'azureml-blobstore-6184be4d-8f14-4f56-b031-56a6ec3813fc', 'account_name': 'vincedev4106797923', 'endpoint': 'core.windows.net', 'protocol': 'https'})

In [76]:
aml_ws = ml_client.workspaces.get(workspace_name)

In [77]:
aml_ws.

{}

In [60]:
asset_name = "debug_info"
run_id = "rag_flow_grid_step_3_variant_0_20240417_111404_012615"

def download_artifacts(run_id, asset_name, ws):
    import requests
    import os
    if region == "centraluseuap":
        url = f"https://int.api.azureml-test.ms/history/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/rundata"
    else:
        url = f"https://ml.azure.com/api/{region}/history/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/rundata"
        payload = {
            "runId": run_id,
            "selectRunMetadata": True
        }
    response = requests.post(url, json=payload, headers=ws._auth.get_authentication_header())
    if response.status_code != 200:
        raise Exception(f"Failed to get output asset id for run {run_id} because RunHistory API returned status code {response.status_code}. Response: {response.text}")
    output_asset_id = response.json()["runMetadata"]["outputs"][asset_name]["assetId"]
    if region == "centraluseuap":
        url = f"https://int.api.azureml-test.ms/data/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/dataversion/getByAssetId"
    else:
        url = f"https://ml.azure.com/api/{region}/data/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/dataversion/getByAssetId"
    payload = {
            "value": output_asset_id,
        }
    response = requests.post(url, json=payload, headers=ws._auth.get_authentication_header())
    if response.status_code != 200:
        raise Exception(f"Failed to get asset path for asset id {asset_id} because Data API returned status code {response.status_code}. Response: {response.text}")
    data_uri = response.json()["dataVersion"]["dataUri"]
    relative_path = data_uri.split("/paths/")[-1]

    destination_path = os.path.join("../runs",run_id)
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    
    default_datastore.download(destination_path, prefix=relative_path +'flow_artifacts', overwrite=True)
    

In [59]:
download_artifacts(run_id, asset_name, ws)


promptflow/PromptFlowArtifacts/rag_flow_grid_step_3_variant_0_20240417_111404_012615/
Downloaded promptflow/PromptFlowArtifacts/rag_flow_grid_step_3_variant_0_20240417_111404_012615/flow_artifacts/000000000_000000024.jsonl, 1 files out of an estimated total of 1


In [23]:
default_datastore.download('../tmp', prefix='promptflow/PromptFlowArtifacts/rag_flow_grid_step_3_variant_0_20240417_111404_012615/flow_artifacts', overwrite=True)

Downloaded promptflow/PromptFlowArtifacts/rag_flow_grid_step_3_variant_0_20240417_111404_012615/flow_artifacts/000000000_000000024.jsonl, 1 files out of an estimated total of 1


1

In [26]:
import pandas as pd
from pandas import json_normalize

# Specify the path to your JSONL file
jsonl_file = '/home/vhoudebine/projects/pf-models-evaluation/tmp/promptflow/PromptFlowArtifacts/rag_flow_grid_step_3_variant_0_20240417_111404_012615/flow_artifacts/000000000_000000024.jsonl'

# Load the JSONL file into a pandas DataFrame
df = pd.read_json(jsonl_file, lines=True)

# Display the DataFrame
df.head()
# Flatten the run_info column
df_flattened = json_normalize(df['run_info'])

# Concatenate the flattened DataFrame with the original DataFrame
df = pd.concat([df, df_flattened], axis=1)

# Drop the original run_info column
df = df.drop('run_info', axis=1)

df.head()


,line_number,start_time,end_time,name,description,status,tags,run_id,status,error,...,message_format,inputs.question,inputs.line_number,inputs.llm_connection,output.response,system_metrics.duration,system_metrics.prompt_tokens,system_metrics.completion_tokens,system_metrics.total_tokens,result.response
0,0,2024-04-17 15:17:02.898586,2024-04-17 15:17:04.664327,,,Completed,NaN,rag_flow_grid_step_3_variant_0_20240417_111404...,Completed,None,...,basic,What is the capital of the United States?,0,mistral-large-maas,The capital of the United States is Washington...,1.765741,0,0,0,The capital of the United States is Washington...
1,3,2024-04-17 15:17:02.949075,2024-04-17 15:17:04.882840,,,Completed,NaN,rag_flow_grid_step_3_variant_0_20240417_111404...,Completed,None,...,basic,What is the capital of Spain?,3,mistral-large-maas,The capital of Spain is Madrid. It's known for...,1.933765,0,0,0,The capital of Spain is Madrid. It's known for...
2,1,2024-04-17 15:17:02.966568,2024-04-17 15:17:05.270017,,,Completed,NaN,rag_flow_grid_step_3_variant_0_20240417_111404...,Completed,None,...,basic,What is the capital of France?,1,mistral-large-maas,"The capital of France is Paris. Known as ""The ...",2.303449,0,0,0,"The capital of France is Paris. Known as ""The ..."
3,2,2024-04-17 15:17:02.961668,2024-04-17 15:17:05.715646,,,Completed,NaN,rag_flow_grid_step_3_variant_0_20240417_111404...,Completed,None,...,basic,What is the capital of Mexico?,2,mistral-large-maas,The capital of Mexico is Mexico City. It's one...,2.753978,0,0,0,The capital of Mexico is Mexico City. It's one...


In [42]:
import pprint
import json

# Example JSON
json_data = {"line_number": 2, "run_info": {"run_id": "rag_flow_grid_step_3_variant_0_20240417_111404_012615_2", "status": "Completed", "error": None, "inputs": {"question": "What is the capital of Mexico?", "line_number": 2, "llm_connection": "mistral-large-maas"}, "output": {"response": "The capital of Mexico is Mexico City. It's one of the largest and most populous cities in the world, known for its rich history, vibrant culture, and unique cuisine. Mexico City is also famous for its historic center, known as Zocalo, which is a UNESCO World Heritage Site and houses ancient Aztec temples and Spanish colonial-era buildings."}, "metrics": None, "request": None, "parent_run_id": "rag_flow_grid_step_3_variant_0_20240417_111404_012615", "root_run_id": "rag_flow_grid_step_3_variant_0_20240417_111404_012615", "source_run_id": None, "flow_id": "default_flow_id", "start_time": "2024-04-17T15:17:02.961668Z", "end_time": "2024-04-17T15:17:05.715646Z", "index": 2, "api_calls": [{"name": "flow", "node_name": "flow", "type": "Flow", "start_time": 1713367022.961668, "end_time": 1713367025.715646, "children": [{"name": "my_python_tool", "type": "Function", "inputs": {"conn": "mistral-large-maas", "message": "What is the capital of Mexico?"}, "output": "The capital of Mexico is Mexico City. It's one of the largest and most populous cities in the world, known for its rich history, vibrant culture, and unique cuisine. Mexico City is also famous for its historic center, known as Zocalo, which is a UNESCO World Heritage Site and houses ancient Aztec temples and Spanish colonial-era buildings.", "start_time": 1713367022.974426, "end_time": 1713367025.711773, "error": None, "children": [], "node_name": "llm_node", "parent_id": "", "id": "6ad0dffb-38a2-4d5e-96ee-e68d3c5bc5bb", "function": "my_python_tool", "system_metrics": {}}], "system_metrics": {"duration": 2.753978, "prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0}, "inputs": {"question": "What is the capital of Mexico?", "line_number": 2, "llm_connection": "mistral-large-maas"}, "output": {"response": "The capital of Mexico is Mexico City. It's one of the largest and most populous cities in the world, known for its rich history, vibrant culture, and unique cuisine. Mexico City is also famous for its historic center, known as Zocalo, which is a UNESCO World Heritage Site and houses ancient Aztec temples and Spanish colonial-era buildings."}, "error": None}], "name": "", "description": "", "tags": None, "system_metrics": {"duration": 2.753978, "prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0}, "result": {"response": "The capital of Mexico is Mexico City. It's one of the largest and most populous cities in the world, known for its rich history, vibrant culture, and unique cuisine. Mexico City is also famous for its historic center, known as Zocalo, which is a UNESCO World Heritage Site and houses ancient Aztec temples and Spanish colonial-era buildings."}, "upload_metrics": False, "otel_trace_id": "00000000000000000000000000000000", "message_format": "basic"}, "start_time": "2024-04-17T15:17:02.961668", "end_time": "2024-04-17T15:17:05.715646", "name": "", "description": "", "status": "Completed", "tags": None}

api_calls = json_data.get('run_info').get('api_calls')

nodes = [call.get('children')[0] for call in api_calls]


In [51]:
import pandas as pd

nodes

[{'name': 'my_python_tool',
  'type': 'Function',
  'inputs': {'conn': 'mistral-large-maas',
   'message': 'What is the capital of Mexico?'},
  'output': "The capital of Mexico is Mexico City. It's one of the largest and most populous cities in the world, known for its rich history, vibrant culture, and unique cuisine. Mexico City is also famous for its historic center, known as Zocalo, which is a UNESCO World Heritage Site and houses ancient Aztec temples and Spanish colonial-era buildings.",
  'start_time': 1713367022.974426,
  'end_time': 1713367025.711773,
  'error': None,
  'children': [],
  'node_name': 'llm_node',
  'parent_id': '',
  'id': '6ad0dffb-38a2-4d5e-96ee-e68d3c5bc5bb',
  'function': 'my_python_tool',
  'system_metrics': {}}]

In [49]:
df

,name,type,inputs,output,start_time,end_time,error,children,node_name,parent_id,id,function,system_metrics,execution_time
0,my_python_tool,Function,"{'conn': 'mistral-large-maas', 'message': 'Wha...",The capital of Mexico is Mexico City. It's one...,1.713367e+09,1.713367e+09,None,[],llm_node,,6ad0dffb-38a2-4d5e-96ee-e68d3c5bc5bb,my_python_tool,{},3.000000e-09


In [47]:
json_data

{'line_number': 2,
 'run_info': {'run_id': 'rag_flow_grid_step_3_variant_0_20240417_111404_012615_2',
  'status': 'Completed',
  'error': None,
  'inputs': {'question': 'What is the capital of Mexico?',
   'line_number': 2,
   'llm_connection': 'mistral-large-maas'},
  'output': {'response': "The capital of Mexico is Mexico City. It's one of the largest and most populous cities in the world, known for its rich history, vibrant culture, and unique cuisine. Mexico City is also famous for its historic center, known as Zocalo, which is a UNESCO World Heritage Site and houses ancient Aztec temples and Spanish colonial-era buildings."},
  'metrics': None,
  'request': None,
  'parent_run_id': 'rag_flow_grid_step_3_variant_0_20240417_111404_012615',
  'root_run_id': 'rag_flow_grid_step_3_variant_0_20240417_111404_012615',
  'source_run_id': None,
  'flow_id': 'default_flow_id',
  'start_time': '2024-04-17T15:17:02.961668Z',
  'end_time': '2024-04-17T15:17:05.715646Z',
  'index': 2,
  'api_cal

In [87]:
from flowgrid import utils

In [88]:
utils.download_artifacts("rag_flow_grid_step_1_variant_0_20240419_145900_268384", 
                                 default_datastore, 
                                 "debug_info", 
                                 aml_ws,
                                 subscription_id,
                                 resource_group,
                                 f"./run_outputs/test/run_test")

AttributeError: module 'flowgrid.utils' has no attribute 'download_artifacts'

In [85]:
utils.duplicate_dict({'llm': {'model': {'name': 'gpt2', 'size': 'medium'}, 'connection': 'mistral-large-maas'}})

[{'llm': {'model': {'name': 'gpt2', 'size': 'medium'},
   'connection': 'mistral-large-maas'}}]

In [86]:
sys.path.append('../flowgrid')

In [93]:
folder_path = "/home/vhoudebine/projects/pf-models-evaluation/run_outputs/run_20240419153429/run_rag_flow_grid_step_0_variant_0_20240419_153432_008863"

In [104]:
import glob
import glob
folder_path = '/home/vhoudebine/projects/pf-models-evaluation/run_outputs/run_20240419153429'

#list all jsonl files
jsonl_files = glob.glob(folder_path + '/**/*.jsonl', recursive=True)



In [113]:
jsonl_files

['/home/vhoudebine/projects/pf-models-evaluation/run_outputs/run_20240419153429/run_rag_flow_grid_step_0_variant_0_20240419_153432_008863/rag_flow_grid_step_0_variant_0_20240419_153432_008863/promptflow/PromptFlowArtifacts/rag_flow_grid_step_0_variant_0_20240419_153432_008863/flow_artifacts/000000000_000000024.jsonl',
 '/home/vhoudebine/projects/pf-models-evaluation/run_outputs/run_20240419153429/run_rag_flow_grid_step_3_variant_0_20240419_153432_011985/rag_flow_grid_step_3_variant_0_20240419_153432_011985/promptflow/PromptFlowArtifacts/rag_flow_grid_step_3_variant_0_20240419_153432_011985/flow_artifacts/000000000_000000024.jsonl',
 '/home/vhoudebine/projects/pf-models-evaluation/run_outputs/run_20240419153429/run_rag_flow_grid_step_2_variant_0_20240419_153432_011481/rag_flow_grid_step_2_variant_0_20240419_153432_011481/promptflow/PromptFlowArtifacts/rag_flow_grid_step_2_variant_0_20240419_153432_011481/flow_artifacts/000000000_000000024.jsonl',
 '/home/vhoudebine/projects/pf-models-ev

In [116]:
import pandas as pd

runs_df = pd.DataFrame()
# Specify the path to your JSONL file
for file_path in jsonl_files:
    df = pd.read_json(file_path, lines=True)
    print(df.shape)
    df_flattened = json_normalize(df['run_info'])

    # Concatenate the flattened DataFrame with the original DataFrame
    df = pd.concat([df, df_flattened], axis=1)
    df['run_id'] = file_path.split('/')[-3]

    # Drop the original run_info column
    df = df.drop('run_info', axis=1)

    df.head()

    runs_df = pd.concat([runs_df, df], axis=0)


(4, 8)
(4, 8)
(4, 8)
(4, 8)


In [119]:
runs_df.columns

Index(['line_number', 'start_time', 'end_time', 'name', 'description',
       'status', 'tags', 'run_id', 'status', 'error', 'metrics', 'request',
       'parent_run_id', 'root_run_id', 'source_run_id', 'flow_id',
       'start_time', 'end_time', 'index', 'api_calls', 'name', 'description',
       'tags', 'upload_metrics', 'otel_trace_id', 'message_format',
       'inputs.question', 'inputs.line_number', 'inputs.llm_connection',
       'output.response', 'system_metrics.duration',
       'system_metrics.prompt_tokens', 'system_metrics.completion_tokens',
       'system_metrics.total_tokens', 'result.response'],
      dtype='object')

In [120]:
runs_df['api_calls']

0    [{'name': 'flow', 'node_name': 'flow', 'type':...
1    [{'name': 'flow', 'node_name': 'flow', 'type':...
2    [{'name': 'flow', 'node_name': 'flow', 'type':...
3    [{'name': 'flow', 'node_name': 'flow', 'type':...
0    [{'name': 'flow', 'node_name': 'flow', 'type':...
1    [{'name': 'flow', 'node_name': 'flow', 'type':...
2    [{'name': 'flow', 'node_name': 'flow', 'type':...
3    [{'name': 'flow', 'node_name': 'flow', 'type':...
0    [{'name': 'flow', 'node_name': 'flow', 'type':...
1    [{'name': 'flow', 'node_name': 'flow', 'type':...
2    [{'name': 'flow', 'node_name': 'flow', 'type':...
3    [{'name': 'flow', 'node_name': 'flow', 'type':...
0    [{'name': 'flow', 'node_name': 'flow', 'type':...
1    [{'name': 'flow', 'node_name': 'flow', 'type':...
2    [{'name': 'flow', 'node_name': 'flow', 'type':...
3    [{'name': 'flow', 'node_name': 'flow', 'type':...
Name: api_calls, dtype: object

In [126]:
pf.runs.get('rag_flow_grid_step_0_variant_0_20240419_153432_008863').properties

{'azureml.promptflow.runtime_name': 'automatic',
 'azureml.promptflow.inputs_mapping': '{"question":"${data.question}"}',
 'azureml.promptflow.disable_trace': 'false',
 'azureml.promptflow.session_id': '77befb68e3670e449c94c2019b3ffe069ce86d08ba6a0cd2',
 'azureml.promptflow.definition_file_name': 'flow.dag.yaml',
 'azureml.promptflow.flow_lineage_id': 'e24a37f34d10f789a58dd9cda30c9c64e5885ebb5a72434a6a32aa67089d78f1',
 'azureml.promptflow.flow_definition_datastore_name': 'workspaceblobstore',
 'azureml.promptflow.flow_definition_blob_path': 'LocalUpload/2cc2c00ae7b79262f06b81cf6923abba/rag_flow_grid_step_0/flow.dag.yaml',
 '_azureml.evaluation_run': 'promptflow.BatchRun',
 'azureml.promptflow.snapshot_id': '750686e5-4d69-4aae-82f3-b30491bbc480',
 'azureml.promptflow.runtime_version': '20240411.v4',
 'azureml.promptflow.total_tokens': '132',
 '_azureml.evaluate_artifacts': '[{"path": "instance_results.jsonl", "type": "table"}]'}

In [ ]:
def prepare_and_execute(
    subscription_id,
    build_id, stage,
    run_id,
    data_purpose,
    flow_to_execute
):
    """
    Run the evaluation loop by executing evaluation flows.

    reads latest evaluation data assets
    executes evaluation flow against each provided bulk-run
    executes the flow creating a new evaluation job
    saves the results in both csv and html format

    Returns:
        None
    """
    main_config = open(f"{flow_to_execute}/llmops_config.json")
    model_config = json.load(main_config)

    for obj in model_config["envs"]:
        if obj.get("ENV_NAME") == stage:
            config = obj
            break

    resource_group_name = config["RESOURCE_GROUP_NAME"]
    workspace_name = config["WORKSPACE_NAME"]
    data_mapping_config = f"{flow_to_execute}/configs/mapping_config.json"
    standard_flow_path = config["STANDARD_FLOW_PATH"]
    data_config_path = f"{flow_to_execute}/configs/data_config.json"

    runtime = config["RUNTIME_NAME"]
    eval_flow_path = config["EVALUATION_FLOW_PATH"]
    experiment_name = f"{flow_to_execute}_{stage}"

    eval_flows = eval_flow_path.split(",")

    pf = PFClient(
        DefaultAzureCredential(),
        subscription_id,
        resource_group_name,
        workspace_name
    )

    standard_flow = f"{flow_to_execute}/{standard_flow_path}"
    dataset_name = []
    config_file = open(data_config_path)
    data_config = json.load(config_file)
    for elem in data_config["datasets"]:
        if "DATA_PURPOSE" in elem and "ENV_NAME" in elem:
            if (stage == elem["ENV_NAME"] and
                    data_purpose == elem["DATA_PURPOSE"]):

                data_name = elem["DATASET_NAME"]
                related_data = elem["RELATED_EXP_DATASET"]
                data = pf.ml_client.data.get(name=data_name, label="latest")
                data_id = f"azureml:{data.name}:{data.version}"
                dataset_name.append(
                    {
                        "data_id": data_id,
                        "ref_data": related_data
                    }
                )

    standard_flow_file = f"{standard_flow}/flow.dag.yaml"

    with open(standard_flow_file, "r") as yaml_file:
        yaml_data = yaml.safe_load(yaml_file)

    default_variants = []
    for node_name, node_data in yaml_data.get("node_variants", {}).items():
        node_variant_mapping = {}
        default_variant = node_data["default_variant_id"]
        node_variant_mapping[node_name] = default_variant
        default_variants.append(node_variant_mapping)

    mapping_file = open(data_mapping_config)
    mapping_config = json.load(mapping_file)
    eval_config_node = mapping_config["evaluation"]

    all_eval_df = []
    all_eval_metrics = []

    run_ids = ast.literal_eval(run_id)

    for flow in eval_flows:
        flow = f"{flow_to_execute}/{flow.strip()}"
        dataframes = []
        metrics = []

        flow_name = (flow.split("/")[-1]).strip()
        mapping_node = eval_config_node[flow_name]
        for flow_run in run_ids:
            my_run = pf.runs.get(flow_run)
            run_data_id = my_run.data.replace("azureml:", "")
            run_data_id = run_data_id.split(":")[0]
            for data_item in dataset_name:
                data_n = data_item["data_id"]
                ref_data = data_item["ref_data"]
                if ref_data == run_data_id:
                    data_id = data_n
                    break

            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

            eval_run = Run(
                flow=flow.strip(),
                data=data_id,
                run=my_run,
                column_mapping=mapping_node,
                runtime=runtime,
                # un-comment the resources parameter assignment
                # and update the size of the compute and also
                # comment the runtime parameter assignment to
                # enable automatic runtime.
                # Reference: COMPUTE_RUNTIME
                # resources={"instance_type": "Standard_E4ds_v4"},
                name=f"{experiment_name}_eval_{timestamp}",
                display_name=f"{experiment_name}_eval_{timestamp}",
                tags={"build_id": build_id},
            )
            eval_run._experiment_name = experiment_name
            eval_job = pf.runs.create_or_update(eval_run, stream=True)
            df_result = None

            time.sleep(15)

            if eval_job.status == "Completed" or eval_job.status == "Finished":
                logger.info(eval_job.status)
                df_result = pf.get_details(eval_job)
                metric_variant = pf.get_metrics(eval_job)

                if (
                    my_run.properties.get(
                        "azureml.promptflow.node_variant",
                        None) is not None
                ):
                    variant_id = \
                        my_run.properties["azureml.promptflow.node_variant"]
                    start_index = variant_id.find("{") + 1
                    end_index = variant_id.find("}")
                    variant_value = \
                        variant_id[start_index:end_index].split(".")

                    df_result[variant_value[0]] = variant_value[1]
                    metric_variant[variant_value[0]] = variant_value[1]
                    df_result["dataset"] = data_id
                    metric_variant["dataset"] = data_id

                    for var in default_variants:
                        for key in var.keys():
                            if key == variant_value[0]:
                                pass
                            else:
                                df_result[key] = var[key]
                                metric_variant[key] = var[key]

                dataframes.append(df_result)
                metrics.append(metric_variant)

                logger.info(json.dumps(metrics, indent=4))
                logger.info(df_result.head(10))

            else:
                raise Exception("Sorry, exiting job with failure..")

        if not os.path.exists("./reports"):
            os.makedirs("./reports")

        combined_results_df = pd.concat(dataframes, ignore_index=True)
        combined_metrics_df = pd.DataFrame(metrics)
        combined_results_df["flow_name"] = flow_name
        combined_metrics_df["flow_name"] = flow_name
        combined_results_df["exp_run"] = flow_run
        combined_metrics_df["exp_run"] = flow_run

        combined_results_df.to_csv(f"./reports/{run_data_id}_result.csv")
        combined_metrics_df.to_csv(f"./reports/{run_data_id}_metrics.csv")

        styled_df = combined_results_df.to_html(index=False)

        with open(f"reports/{run_data_id}_result.html", "w") as c_results:
            c_results.write(styled_df)

        html_table_metrics = combined_metrics_df.to_html(index=False)
        with open(f"reports/{run_data_id}_metrics.html", "w") as c_metrics:
            c_metrics.write(html_table_metrics)

        all_eval_df.append(combined_results_df)
        all_eval_metrics.append(combined_metrics_df)

    final_results_df = pd.concat(all_eval_df, ignore_index=True)
    final_metrics_df = pd.concat(all_eval_metrics, ignore_index=True)
    final_results_df["stage"] = stage
    final_results_df["experiment_name"] = experiment_name
    final_results_df["build"] = build_id

    final_results_df.to_csv(f"./reports/{experiment_name}_result.csv")
    final_metrics_df.to_csv(f"./reports/{experiment_name}_metrics.csv")

    styled_df = final_results_df.to_html(index=False)
    with open(f"reports/{experiment_name}_result.html", "w") as f_results:
        f_results.write(styled_df)

    html_table_metrics = final_metrics_df.to_html(index=False)
    with open(f"reports/{experiment_name}_metrics.html", "w") as f_metrics:
        f_metrics.write(html_table_metrics)


In [131]:
from promptflow.entities import Run
import datetime 

data="azureml://subscriptions/6c065ea7-65cd-4a34-8e2a-3e21ad4a8e9f/resourcegroups/vince-rg/workspaces/vince-dev/datastores/workspaceblobstore/paths/promptflow/PromptFlowArtifacts/rag_flow_grid_step_4_variant_0_20240422_112227_503149/instance_results.jsonl"
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_name = "rag_flow_grid_step_4_eval"
eval_run = Run(
                flow='../flows/evaluation_flow',
                data='../evaluation_data/data.jsonl',
                run="rag_flow_grid_step_4_variant_0_20240422_112227_503149",
                column_mapping={
                    "answer": "${run.outputs.response}",
                    "ground_truth": "${data.ground_truth}"         },
                # un-comment the resources parameter assignment
                # and update the size of the compute and also
                # comment the runtime parameter assignment to
                # enable automatic runtime.
                # Reference: COMPUTE_RUNTIME
                # resources={"instance_type": "Standard_E4ds_v4"},
                name=f"{experiment_name}_eval_{timestamp}",
                display_name=f"{experiment_name}_eval_{timestamp}",
            )
#eval_run._experiment_name = experiment_name
eval_job = pf.runs.create_or_update(eval_run, stream=True)

[2024-04-22 12:26:39,661][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ml.azure.com/runs/rag_flow_grid_step_4_eval_eval_20240422_122637?wsid=/subscriptions/6c065ea7-65cd-4a34-8e2a-3e21ad4a8e9f/resourcegroups/vince-rg/providers/Microsoft.MachineLearningServices/workspaces/vince-dev
2024-04-22 16:26:47 +0000     174 promptflow-runtime INFO     [rag_flow_grid_step_4_eval_eval_20240422_122637] Receiving v2 bulk run request 9cb3bddf-8729-4fd9-8996-ac85ed3d1fe7: {"flow_id": "rag_flow_grid_step_4_eval_eval_20240422_122637", "flow_run_id": "rag_flow_grid_step_4_eval_eval_20240422_122637", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "3caa1cc8-7b6e-4e1b-aa90-79c49a46e15a"}, "flow_dag_file": "flow.dag.yaml"}, "log_path": "https://vincedev4106797923.blob.core.windows.net/azureml/ExperimentRun/dcid.rag_flow_grid_step_4_eval_eval_20240422_122637/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=**data_scrubbed**&skoid=4af85c54-3f14-4b6a-a706-a4a6e971c4e7&sktid=16b3c013-d300-468d-ac64-7eda0820b6d3&skt=2024-04-22T13%

In [134]:
pf.get_metrics(eval_job)

{'f1_score': 0.12}

In [137]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

run = pf.run(
            flow='../flows/evaluation_flow',
                data='../evaluation_data/data.jsonl',
                run="rag_flow_grid_step_4_variant_0_20240422_112227_503149",
                column_mapping={
                    "answer": "${run.outputs.response}",
                    "ground_truth": "${data.ground_truth}"         },
                # un-comment the resources parameter assignment
                # and update the size of the compute and also
                # comment the runtime parameter assignment to
                # enable automatic runtime.
                # Reference: COMPUTE_RUNTIME
                # resources={"instance_type": "Standard_E4ds_v4"},
                name=f"{experiment_name}_eval_{timestamp}",
                display_name=f"{experiment_name}_eval_{timestamp}",
        )

pf.stream(run)

[2024-04-22 13:51:50,581][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ml.azure.com/runs/rag_flow_grid_step_4_eval_eval_20240422_135149?wsid=/subscriptions/6c065ea7-65cd-4a34-8e2a-3e21ad4a8e9f/resourcegroups/vince-rg/providers/Microsoft.MachineLearningServices/workspaces/vince-dev
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
2024-04-22 17:56:08 +0000      51 promptflow-runtime INFO     [rag_flow_grid_step_4_eval_eval_20240422_135149] Receiving v2 bulk run request ab41a9e9-b723-4c63-b793-40a506eef9e6: {"flow_id": "rag_flow_grid_step_4_eval_eval_20240422_135149", "flow_run_id": "rag_flow_grid_step_4_eval_eval_20240422_135149", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "f03ff8db-37ea-448f-b10e

In [1]:
e = raw_input("what is your name")

NameError: name 'raw_input' is not defined